# Decomposition and Products — Notes Companion

**6.7970/8.750 Symmetry and its Application to Machine Learning**

This notebook accompanies the [Decomposition and Products notes](https://symm4ml.mit.edu/notes/decomposition-and-products). Run these examples to decompose representations into irreps and explore tensor products.

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/atomicarchitects/symm4ml-colabs/blob/main/decomposition_notes_companion.ipynb)

## Setup

In [ ]:
%%capture
!pip install https://symm4ml.mit.edu/_static/symm4ml_s26/symm4ml/symm4ml_latest.zip

In [ ]:
import numpy as np
from symm4ml import groups, linalg, rep

## Reference: $P(3)$ data

In [ ]:
# Build P(3) table and find all irreps
p3_matrices = groups.permutation_matrices(3)
table = groups.make_multiplication_table(p3_matrices)
irreps = rep.infer_irreps(table)

labels = [f"\u0393_{i+1}" for i in range(len(irreps))]
print(f"P(3): h = {len(table)}, {len(irreps)} irreps")
for label, ir in zip(labels, irreps):
    print(f"  {label}: dimension {ir.shape[1]}")

---
## Example: Decomposing a Scrambled Representation

The decomposition algorithm works even when the block structure is **hidden** by a random change of basis.
Let's build $\Gamma_1 \oplus \Gamma_2$, scramble it with a random unitary, and verify that `decompose_rep_into_irreps` recovers the same irreps.

In [ ]:
# Build the direct sum Γ_1 ⊕ Γ_2 (block diagonal)
gamma_1d = [ir for ir in irreps if ir.shape[1] == 1][0]  # a 1D irrep
gamma_2d = [ir for ir in irreps if ir.shape[1] == 2][0]  # the 2D irrep

direct = rep.direct_sum(gamma_1d, gamma_2d)
print(f"Direct sum shape: {direct.shape}")
print(f"D(g=1) (block diagonal):\n{direct[1]}")

In [ ]:
# Scramble with a random unitary
np.random.seed(123)
d = direct.shape[1]
U, _ = np.linalg.qr(np.random.randn(d, d))
scrambled = rep.similarity_transform(direct, U)

print(f"Scrambled D(g=1) (no visible block structure):\n{np.round(scrambled[1], 4)}")

In [ ]:
# Decompose the scrambled representation
recovered = rep.decompose_rep_into_irreps(scrambled)

print(f"Recovered {len(recovered)} irreps:")
for i, sub in enumerate(recovered):
    for j, known in enumerate(irreps):
        if rep.are_isomorphic(sub, known):
            print(f"  Sub-irrep {i+1} \u2245 {labels[j]} (dim {known.shape[1]})")
            break

print("\nThe algorithm sees through the scrambling!")

---
## Example: Tensor Product Decomposition Table

The **tensor product** (Kronecker product) of two representations $\Gamma_i \otimes \Gamma_j$ is a new representation of dimension $\ell_i \cdot \ell_j$.
It is generally reducible and can be decomposed into irreps — this is the **Clebsch–Gordan problem**.

Let's compute the full "multiplication table" of irreps for $P(3)$.

In [ ]:
# Compute all tensor products and decompose each
n_irreps = len(irreps)

print("Tensor product decomposition table for P(3):\n")
header = "          " + "  ".join(f"{labels[j]:>8s}" for j in range(n_irreps))
print(header)
print("          " + "-" * (10 * n_irreps))

for i in range(n_irreps):
    row = f"{labels[i]:>8s}  "
    for j in range(n_irreps):
        # Compute tensor product
        tp = rep.tensor_product(irreps[i], irreps[j])
        # Decompose into irreps
        sub = rep.decompose_rep_into_irreps(tp)
        # Identify each sub-irrep
        parts = []
        for s in sub:
            for k, known in enumerate(irreps):
                if rep.are_isomorphic(s, known):
                    parts.append(labels[k])
                    break
        row += f"{'\u2295'.join(parts):>10s}"
    print(row)